# Connecting with the Credo AI Governance Platform

## Overview
Connecting with the Credo AI Governance Platform is straightforward using Lens. After following this tutorial you'll be up and running!

### Config File
First, ensure you have a `config` file somewhere on your computer. The default location where Lens will look for it is `~/.credoconfig`. The structure of the config should look like this

```
TENANT={tenant name} # Example: credoai
CREDO_URL=<your credo url>  # Example: https://api.credo.ai 
API_KEY=<your api key> # Example: JSMmd26...
```

This config gives Lens API access to the Governance Platform.

### Connecting to a particular model, dataset, or AI Solution

Lens connects to the Governance Platform via a `CredoGovernance` object. This specifies the IDs of the AI Solution, model and data you want to associate your assessments with. The IDs are found in the url of your AI solution or model.

For instance, the AI Solution ID is found here: `...credo.com/ai-solutions/{ai_solution_id}`

And the model Id is found here: `...credo.com/models/{model_id}`

## Train your Model

Some quick setup... This is all of your datascience work before assessment and integration with Credo AI.

In [ ]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier

# Base Lens imports
import credoai.lens as cl

In [ ]:
data = fetch_creditdefault()
X = data['data'].drop(columns=['SEX'])
y = data['target']
sensitive_feature = data['data']['SEX']

model = GradientBoostingClassifier()
model.fit(X,y)

## Using Lens

This is going to be familiar if you've gone through our quickstart tutorial! The only addition is the use of a `CredoGovernance` object.

The CredoGovernance object gives Lens a few features.

* The ability to get an alignment spec from the AI Solution (created during the aligned process). This alignment spec can be supplemented (or overwritten) with an alignment spec you define in code and pass to Lens.
* The ability to easily export metrics and reports.

\* **Note** \* The metrics and reports will _either_ be exported to the model and data specified in by `CredoGovernance` (using the `model_id` and `data_id` arguments) OR a new model and data will be registered on your CredoGovernance platform automatically, which will then be used for export.

With that said, let's see how this works in practice.

In [ ]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          X=X, 
                          y=y.astype(int),
                          sensitive_features=sensitive_feature)

# NOTE! This extra artifact is needed to connect to Credo AI's Governance Platform
credo_governance = cl.CredoGovernance(ai_solution_id="your-ai-solution-id",
                                      model_id="your-model-id"
                                     )

In [ ]:
# specify the metrics that will be used by the FairnessBase assessment. 
# This spec will supplement (or overide!) whatever spec is 
# pulled down from the Governance Platform
alignment_spec = {
    'FairnessBase': {'metrics': ['precision_score']}
}

#### Run Lens

In addition to the model and data, we can pass the CredoGovernance object. This is how Lens knows which AI solution, model and/or data to interacts with. If you are running assessments on multiple models, you'll create a separate `CredoGovernance` object (and separate Lens instances) for each.

That's it!

In [ ]:
from credoai.assessment import FairnessBaseAssessment
lens = cl.Lens(model=credo_model,
               governance=credo_governance,
               assessments = [FairnessBaseAssessment()],
               data=credo_data,
               spec=alignment_spec)

In [ ]:
# just add the export option and the metrics will be sent to the Governance Platform.
lens.run_assessments(export=True)

In [ ]:
# Reports can be exported too!
lens.create_reports(export=True)

## Explicitly registering model and data artifacts

While Lens will take care of registering a model and data for you if no `model_id` or `data_id` is provided, you can also manually register models yourself. 

This is helpful if you want to run Lens multiple times. First you would register a new model and/or data, and then used the returned IDs in a CredoGovernance object.

In [6]:
import credoai.integration as ci

### Registering a Project and Model(s)

Models are organized under Projects. If you register a Model, a Project will be automatically created to house the Model.

If a Project already exists that you'd like to use you can supply the Project ID to the `project_id` argument.

In [7]:
ids = ci.register_model(model_name='my model')
ids

{'name': 'my model',
 'model_id': '354hLxUhBsub6x5KgriBtc',
 'project_id': '8ENSXBCyyw9B7ZNjB9VSoU'}

In [9]:
project_id = ids['project_id']
second_ids = ci.register_model(model_name='another model', project_id=project_id)

You can also register a project first and use the returned `project_id` to pass to registering models.

In [11]:
project_id = ci.register_project('my project')
project_id

{'name': 'my project', 'project_id': 'cUFdwqwncbGVkzTbwoaQBD'}